In [ ]:
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report

# Load CIFAR-10 dataset
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Normalize pixel values
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

# One-hot encode the labels
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

# Sequential model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(10, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(x_train, y_train, epochs=10, batch_size=64, validation_data=(x_test, y_test))

# Evaluate the model
test_loss, test_acc = model.evaluate(x_test, y_test)
print("Test Accuracy:", test_acc)

# Pretrained model on ImageNet
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(32, 32, 3))

# Freeze the layers
for layer in base_model.layers:
    layer.trainable = False

# Add new classifier layers
x = Flatten()(base_model.output)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
output = Dense(10, activation='softmax')(x)

# New model
transfer_model = tf.keras.Model(base_model.input, output)

# Compile the model
transfer_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
transfer_model.fit(x_train, y_train, epochs=10, batch_size=64, validation_data=(x_test, y_test))

# Evaluate the model
test_loss, test_acc = transfer_model.evaluate(x_test, y_test)
print("Transfer Learning Test Accuracy:", test_acc)


170498071/170498071 [==============================] - 3s 0us/step
Epoch 1/10
782/782 [==============================] - 66s 82ms/step - loss: 1.6661 - accuracy: 0.3868 - val_loss: 1.3535 - val_accuracy: 0.5142
Epoch 2/10
782/782 [==============================] - 60s 77ms/step - loss: 1.3023 - accuracy: 0.5341 - val_loss: 1.1551 - val_accuracy: 0.5923
Epoch 3/10
782/782 [==============================] - 61s 78ms/step - loss: 1.1561 - accuracy: 0.5951 - val_loss: 1.0656 - val_accuracy: 0.6259
Epoch 4/10
782/782 [==============================] - 61s 78ms/step - loss: 1.0554 - accuracy: 0.6310 - val_loss: 0.9813 - val_accuracy: 0.6551
Epoch 5/10
782/782 [==============================] - 62s 79ms/step - loss: 0.9682 - accuracy: 0.6637 - val_loss: 0.9135 - val_accuracy: 0.6813
Epoch 6/10
782/782 [==============================] - 62s 79ms/step - loss: 0.9110 - accuracy: 0.6835 - val_loss: 0.9052 - val_accuracy: 0.6862
Epoch 7/10
782/782 [==============================] - 65s 83ms/step -